In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san-pedro,4.7485,-6.6363,27.45,80,97,4.69,CI,1682106635
1,1,margaret river,-33.9500,115.0667,13.58,92,11,4.02,AU,1682106636
2,2,ar rumaythah,31.5263,45.2025,25.83,35,11,5.41,IQ,1682106637
3,3,pandan,14.0453,124.1694,25.65,85,94,2.46,PH,1682106637
4,4,mana,5.6592,-53.7767,26.02,88,100,3.69,GF,1682106638


**Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.**

In [5]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 1,
    color = 'City'
)

# Display the map
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

**Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition**

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
vacay_cities_df = city_data_df[(city_data_df['Cloudiness'] <= 25) &
              (city_data_df['Max Temp'] >= 19) & 
              (city_data_df['Max Temp'] <= 29) &
              (city_data_df['Wind Speed'] <= 6)]

# Drop any rows with null values
vacay_cities_df = vacay_cities_df.dropna()

# Display sample data
vacay_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,ar rumaythah,31.5263,45.2025,25.83,35,11,5.41,IQ,1682106637
24,24,saint-joseph,-21.3667,55.6167,25.21,79,23,3.20,RE,1682106647
46,46,bamboo flat,11.7000,92.7167,28.00,71,22,4.07,IN,1682106658
50,50,avarua,-21.2078,-159.7750,26.03,78,19,4.12,CK,1682106660
66,66,albany,42.6001,-73.9662,25.96,33,14,2.24,US,1682106670


**Step 3: Create a new DataFrame called hotel_df**

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacay_cities_df.copy()

hotel_df = hotel_df[['City_ID', 'City', 'Lat', 'Lng', 'Humidity', 'Country']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
2,2,ar rumaythah,31.5263,45.2025,35,IQ,
24,24,saint-joseph,-21.3667,55.6167,79,RE,
46,46,bamboo flat,11.7000,92.7167,71,IN,
50,50,avarua,-21.2078,-159.7750,78,CK,
66,66,albany,42.6001,-73.9662,33,US,


**Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.**

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories': 'accommodation.hotel',
          'filter': "",
          'bias': "",
          'limit': 5,
          'apiKey': geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]

    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df 

Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle%3A45.2025%2C31.5263%2C10000&bias=proximity%3A45.2025%2C31.5263&limit=5&apiKey=599c4f95e351432494c8f35770bf70f9
ar rumaythah - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle%3A55.6167%2C-21.3667%2C10000&bias=proximity%3A55.6167%2C-21.3667&limit=5&apiKey=599c4f95e351432494c8f35770bf70f9
saint-joseph - nearest hotel: PALM Hotel & Spa
https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle%3A92.7167%2C11.7%2C10000&bias=proximity%3A92.7167%2C11.7&limit=5&apiKey=599c4f95e351432494c8f35770bf70f9
bamboo flat - nearest hotel: Hotel Megapode Nest
https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle%3A-159.775%2C-21.2078%2C10000&bias=proximity%3A-159.775%2C-21.2078&limit=5&apiKey=599c4f95e351432494c8f35770bf70f9
avarua - nearest hotel: Paradise Inn
https://api.geoapify.com/v2/places?c

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
2,2,ar rumaythah,31.5263,45.2025,35,IQ,No hotel found
24,24,saint-joseph,-21.3667,55.6167,79,RE,PALM Hotel & Spa
46,46,bamboo flat,11.7000,92.7167,71,IN,Hotel Megapode Nest
50,50,avarua,-21.2078,-159.7750,78,CK,Paradise Inn
66,66,albany,42.6001,-73.9662,33,US,No hotel found
...,...,...,...,...,...,...,...
548,548,port-vila,-17.7338,168.3219,94,VU,Holiday Inn Resort Vanuatu
555,555,grand gaube,-20.0064,57.6608,79,MU,Coco Villa
557,557,kirakira,-10.4544,161.9205,83,SB,No hotel found
571,571,coquimbo,-29.9533,-71.3436,56,CL,Hotel Iberia


**Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.**

In [35]:
%%capture --no-display

# Configure the map plot
map_plot_humidity_hotel = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 600,
    frame_height = 500,
    size = 'Humidity',
    scale = 1.5,
    color = 'City',
    hover_cols = 'all')

# Display the map
map_plot_humidity_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,City_ID,Country,Hotel Name)